In [66]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import (
    ImageDataGenerator, 
    array_to_img, 
    img_to_array, 
    load_img
)
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, accuracy_score


### Understanding `ImageDataGenerator` for Data Augmentation

When training deep learning models, especially in computer vision tasks, having a large dataset is crucial for improving model generalization and performance. However, collecting a large dataset can be challenging. This is where **data augmentation** helps.

#### What is `ImageDataGenerator`?
The `ImageDataGenerator` class in Keras provides a way to augment image data by applying random transformations to images in each batch during training. This helps improve model robustness and prevents overfitting.

#### Code Breakdown:

Each parameter in `ImageDataGenerator` applies a specific transformation:

- `rotation_range=40`: Rotates images randomly within a range of ±40 degrees.
- `width_shift_range=0.2`: Shifts the image horizontally by up to 20% of the image width.
- `height_shift_range=0.2`: Shifts the image vertically by up to 20% of the image height.
- `rescale=1./255`: Normalizes pixel values by scaling them between 0 and 1.
- `shear_range=0.2`: Applies a shear transformation with a magnitude of 20%.
- `zoom_range=0.2`: Zooms into or out of the image by up to 20%.
- `horizontal_flip=True`: Randomly flips images horizontally.
- `fill_mode='nearest'`: Determines how pixels are filled in after transformations (nearest neighbor interpolation in this case).

In [67]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
)

Load Images using train_generator

In [ ]:
anger_gen = datagen.flow_from_directory(
    directory=r'C:\RPI\Graduation Semester\Behavioral Data Science [PSYC 4961]\Homework\Homework 3\mery',
    classes=['mery_anger'],
    class_mode='binary',
    target_size=(224,224),
    batch_size=800,
    shuffle=True
)

joy_gen = datagen.flow_from_directory(
    directory=r'C:\RPI\Graduation Semester\Behavioral Data Science [PSYC 4961]\Homework\Homework 3\mery',
    classes=['mery_joy'],
    class_mode='binary',
    target_size=(224,224),
    batch_size=800,
    shuffle=True
)

# Grabs 800 Of Each
x_a, y_a = next(anger_gen)   # y_a All Zeros
x_j, y_j = next(joy_gen)     # y_j All Zeros

# FORCE Joy Labels To 1
y_j = np.ones_like(y_j)

# Stacks and Shuffles Into One Batch Of 1600
x_batch = np.concatenate([x_a, x_j], axis=0)
y_batch = np.concatenate([y_a, y_j], axis=0)
idx     = np.random.permutation(len(x_batch))
x_batch, y_batch = x_batch[idx], y_batch[idx]

print("Raw labels count:", np.bincount(y_batch.astype(int))) 

Found 1428 images belonging to 1 classes.
Found 1140 images belonging to 1 classes.
Raw labels count: [800 800]


### Building the Convolutional Neural Network (CNN)

In [ ]:
# One Hot Encode
y_train = to_categorical(y_batch, num_classes=2)
x_train = x_batch

# Split Into Train/Validation
x_tr, x_val, y_tr, y_val = train_test_split(
    x_train, y_train,
    test_size=0.2,
    random_state=4,
    shuffle=True
)

# Build CNN With Dropout
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(2, activation='softmax')
])

# Compile With Categorical Cross‑Entropy
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

c:\Users\maxim\.conda\envs\behavioral-data-science\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_21 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,218 (42.61 MB)

 Trainable params: 11,169,218 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

### Training the Network

In [70]:
# Train
history = model.fit(
    x_tr, y_tr,
    epochs=15,
    batch_size=32,
    validation_data=(x_val, y_val)
)

Epoch 1/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 20s 461ms/step - accuracy: 0.4937 - loss: 0.8517 - val_accuracy: 0.5063 - val_loss: 0.6931
Epoch 2/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 18s 437ms/step - accuracy: 0.5204 - loss: 0.6930 - val_accuracy: 0.5375 - val_loss: 0.6919
Epoch 3/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 18s 446ms/step - accuracy: 0.5550 - loss: 0.6902 - val_accuracy: 0.5562 - val_loss: 0.6894
Epoch 4/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 17s 436ms/step - accuracy: 0.5879 - loss: 0.6706 - val_accuracy: 0.8438 - val_loss: 0.5140
Epoch 5/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 18s 440ms/step - accuracy: 0.8412 - loss: 0.4360 - val_accuracy: 0.8750 - val_loss: 0.2876
Epoch 6/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 18s 446ms/step - accuracy: 0.9409 - loss: 0.1714 - val_accuracy: 0.9094 - val_loss: 0.1980
Epoch 7/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 18s 453ms/step - accuracy: 0.9716 - loss: 0.0925 - val_accuracy: 0.9781 - val_loss: 0.0634
Epoch 8/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 24s 603ms/step - accuracy: 0.9951 - loss: 0.0206 - val_accu

### Test The Model

In [ ]:
# Build A Pure‐Test Datagen (No Augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

# Build Two Separate Test Generators, 200 Images Each
test_anger_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    directory   = r'C:\RPI\Graduation Semester\Behavioral Data Science [PSYC 4961]\Homework\Homework 3\mery',
    classes     = ['mery_anger'],
    class_mode  = 'binary',
    target_size = (224,224),
    batch_size  = 200,
    shuffle     = True
)
test_joy_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    directory   = r'C:\RPI\Graduation Semester\Behavioral Data Science [PSYC 4961]\Homework\Homework 3\mery',
    classes     = ['mery_joy'],
    class_mode  = 'binary',
    target_size = (224,224),
    batch_size  = 200,
    shuffle     = True
)

# Grabs One Batch Of Each
x_a, y_a = next(test_anger_gen)
x_j, y_j = next(test_joy_gen)

# FORCE Joy Labels To 1
y_j = np.ones_like(y_j)

# Stack Them
x_test = np.concatenate([x_a, x_j], axis=0)
y_test = np.concatenate([y_a, y_j], axis=0)

# Shuffle The Combined Batch
idx = np.random.permutation(len(x_test))
x_test, y_test = x_test[idx], y_test[idx]

print("Label counts:", np.bincount(y_test.astype(int)))

# Predict
pred_probs = model.predict(x_test)
y_pred     = np.argmax(pred_probs, axis=1)

# Compute Accuracy
acc = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {acc*100:.2f}%")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:")
print(cm)

Found 1428 images belonging to 1 classes.
Found 1140 images belonging to 1 classes.
Label counts: [200 200]
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step
Test accuracy: 99.25%
Confusion matrix:
[[200   0]
 [  3 197]]
